In [4]:
import os
from getpass import getpass

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Google Drive already mounted")

token = getpass('Enter your GitHub token: ')

!git clone https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git

!git pull https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git

!cp /content/drive/MyDrive/NBA_Fantasy_Clustering.ipynb /content/NBA-Fantasy-Points-Prediction/notebooks/

%cd /content/NBA-Fantasy-Points-Prediction

!git add notebooks/NBA_Fantasy_Clustering.ipynb

!git commit -m "Add clustering notebook for NBA Fantasy project"

!git push https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git


Google Drive already mounted
Enter your GitHub token: ··········
fatal: destination path 'NBA-Fantasy-Points-Prediction' already exists and is not an empty directory.
From https://github.com/sammig6i/NBA-Fantasy-Points-Prediction
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
/content/NBA-Fantasy-Points-Prediction
On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	NBA-Fantasy-Points-Prediction/

nothing added to commit but untracked files present (use "git add" to track)
Everything up-to-date
